In [ ]:
!pip install pip -qU
!pip install -r requirements.txt -q

In [ ]:
import tensorflow as tf
import mlflow
import mlflow.tensorflow
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

# # Load the ResNet50 model
# resnet_model = ResNet50(weights='imagenet')

loaded_model = tf.keras.models.load_model("tensorflow/model")


In [ ]:
# Function to preprocess an image for ResNet50
def preprocess_image(img_path):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = preprocess_input(img_array)
    img_array = tf.expand_dims(img_array, 0)
    return img_array

# Save the ResNet50 model in TensorFlow SavedModel format
@tf.function(input_signature=[tf.TensorSpec(shape=[None, 224, 224, 3], dtype=tf.float32)])
def resnet_serving_fn(images):
    return {"predictions": resnet_model(images)}

In [ ]:
import os
import json
import tempfile
import mlflow
MLFLOW_ROUTE = os.getenv("MLFLOW_ROUTE")
mlflow.set_tracking_uri(MLFLOW_ROUTE)
mlflow.set_experiment("resnet")
mlflow.pytorch.autolog(registered_model_name="resnet")


In [ ]:
# Log the ResNet50 model with a signature to MLflow
with mlflow.start_run() as run:
    mlflow.tensorflow.log_model(
        model=loaded_model,
        artifact_path="model"
    )
    
    temp_model_dir = tempfile.TemporaryDirectory().name + "/tensorflow-model"
    loaded_model.save(temp_model_dir)
    mlflow.log_artifact(temp_model_dir)